In [4]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!pip install selenium chromedriver_autoinstaller

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 39 n

Install web driver: https://medium.com/@MinatoNamikaze02/running-selenium-on-google-colab-a118d10ca5f8

%%shell \\
sudo apt -y update \\
sudo apt install -y wget curl unzip \\
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb \\
dpkg -i libu2f-udev_1.1.4-1_all.deb \\
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb \\

wget -N https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/118.0.5993.70/linux64/chromedriver-linux64.zip -P /tmp/ \\
unzip -o /tmp/chromedriver-linux64.zip -d /tmp/ \\
chmod +x /tmp/chromedriver-linux64/chromedriver \\
mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver \\
pip install selenium chromedriver_autoinstaller


In [5]:
# import libraries
import sys
from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [6]:
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # this is must
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chromedriver_autoinstaller.install()

In [7]:
def scrape_links_to_DON(driver, page_num):
    """Scrape links to webpages of disease output news(DON).
    # the class-tags may be different from time to time. Hence, check different tags.
    Parameters
    ----------
    driver : selenium.Webdriver
        webdriver for website interactions
    Returns
    -------
    : list
        urls to webpage
    """

    all_DON_links = []

    for i in range(page_num):
        all_links_to_details = driver.find_elements(By.CLASS_NAME, "sf-list-vertical__item")

        links = [link.get_attribute("href") for link in all_links_to_details]

        all_DON_links.extend(links)

        # Click the next page button
        try:
            textbox = driver.find_element(By.CLASS_NAME, "k-textbox")
            print(f"page {textbox.get_attribute('aria-label')}") #print out which page we are on

            next_page_button = driver.find_elements(By.CLASS_NAME, "k-link.k-pager-nav")[2]
            next_page_button.click()

            time.sleep(3)

        except:
            print("No more pages")
            break

    return all_DON_links

In [8]:
# based on: https://medium.com/@jodancker/scraping-a-dynamically-loading-website-with-selenium-a399b076f79a
urlpage = "https://www.who.int/zh/emergencies/disease-outbreak-news"

# open the driver
driver = webdriver.Chrome(options=chrome_options)
driver.get(urlpage)
driver.implicitly_wait(100)

In [9]:
links_to_DONS = scrape_links_to_DON(driver, 12)

page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11
page 12
No more pages


In [10]:
print(links_to_DONS)
print(len(links_to_DONS))

['https://www.who.int/zh/emergencies/disease-outbreak-news/item/2023-DON488', 'https://www.who.int/zh/emergencies/disease-outbreak-news/item/2023-DON489', 'https://www.who.int/zh/emergencies/disease-outbreak-news/item/2023-DON487', 'https://www.who.int/zh/emergencies/disease-outbreak-news/item/2023-DON485', 'https://www.who.int/zh/emergencies/disease-outbreak-news/item/2023-DON486', 'https://www.who.int/zh/emergencies/disease-outbreak-news/item/2023-DON484', 'https://www.who.int/zh/emergencies/disease-outbreak-news/item/2023-DON483', 'https://www.who.int/zh/emergencies/disease-outbreak-news/item/2023-DON482', 'https://www.who.int/zh/emergencies/disease-outbreak-news/item/2023-DON481', 'https://www.who.int/zh/emergencies/disease-outbreak-news/item/2023-DON479', 'https://www.who.int/zh/emergencies/disease-outbreak-news/item/2023-DON480', 'https://www.who.int/zh/emergencies/disease-outbreak-news/item/2023-DON477', 'https://www.who.int/zh/emergencies/disease-outbreak-news/item/2023-DON478'

In [21]:
# write links_to_DONS in a csv file
import csv
with open('/content/drive/MyDrive/Colab Notebooks/Big_data/links_to_DONS.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    for link in links_to_DONS:
        writer.writerow([link])

In [11]:
#quit driver for scrape the links to DONs
driver.quit()

## Scrape text for each news in both Chinese and English

In [12]:
# test field
news_url = links_to_DONS[3]
print(news_url)

https://www.who.int/zh/emergencies/disease-outbreak-news/item/2023-DON485


In [13]:
# open the driver
cn_driver = webdriver.Chrome(options=chrome_options)
cn_driver.get(news_url)
cn_driver.implicitly_wait(15)

In [14]:
elements = cn_driver.find_elements(By.TAG_NAME, "p")
text = ""
for element in elements:
    element_text = element.text
    text += element_text

In [15]:
text

'自2023年7月2日结束的流行病学第26周以来，尼日利亚多个州的白喉病例出现异常增加。从2023年6月30日到8月31日，11个州的59个地方政府辖区总共报告了5898例疑似病例。在第34周（结束于2023年8月27日）当中，五个州的20个地方政府辖区报告了234例疑似病例，在采集的22份样本中有1例实验室确诊病例1。这些病例中有18例存在流行病学关联2，141例被归类为符合临床标准的病例。白喉是一种高度传染性的疫苗可预防疾病，主要由白喉棒状杆菌引起，在5%-10%的病例中可能致命，幼儿死亡率更高。世卫组织对尼日利亚白喉疫情的最新风险评估认为，国家一级仍然存在高风险，区域和全球一级风险较低。为应对疫情，正在与世卫组织和其他伙伴合作，并在尼日利亚疾病控制中心协调下实施公共卫生措施，如接种疫苗、加强监测以便及早发现病例、管理病例和沟通风险等。自上一次于2023年4月27日发布尼日利亚白喉疾病暴发新闻以来，该国每周向世卫组织报告白喉疑似病例。然而，在2023年6月30日至8月31日期间，尼日利亚的确诊白喉病例数量出现异常增加4（图1）。从2023年6月30日到8月31日，11个州的59个地方政府辖区总共报告了5898例疑似病例。报告的大多数（99.4%）疑似病例来自卡诺州（1816例）、卡齐纳州（234例）、约比州（158例）、包奇州（79例）、卡杜纳州（45例）和博尔诺州（33例）。自2022年首次报告疫情以来已累计报告了8353例疑似病例，其中有4717例（56.5%）已得到确诊4（包括实验室确诊病例1（169例；3.6%）、存在流行病学关联的病例2（117例；2.5%）和符合临床标准的病例3（4431例；93.9%））。同时1857例（22.2%）因与白喉症状不符而被排除，1048例（12.5%）尚待归类，731例（8.8%）诊断结果不详。病死率从上次更新时的6.7%略微降至6.1%。在4717例确诊病例中，3466例（73.5%）年龄为1至14岁，其中699例为0-4岁，1505例为5-9岁，1262例为10至14岁。半数以上病例（2656例；56.3%）为女性。确诊病例中只有1074例（22.8%）接种了全部剂量的白喉疫苗，299例（6.3%）接种了部分剂量疫苗。半数以上病例（2801例；59.4%）未接种疫苗。图1. 2022年5月1日至2023年8月27日

In [16]:
cn_driver.quit()